# Choromorph – GIF Generation of Step-by-Step Vector Field
This notebook visualizes the morphing process using POI attraction and neighbor cohesion. It saves each step as frames and compiles them into a GIF.

In [88]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import os
import shutil
import imageio


In [89]:
def build_square_grid(n_side: int = 20):
    x, y = np.meshgrid(np.linspace(0, 1, n_side), np.linspace(0, 1, n_side))
    grid = np.column_stack([x.ravel(), y.ravel()])
    edges = []
    for r in range(n_side):
        for c in range(n_side):
            idx = r * n_side + c
            if c < n_side - 1:
                edges.append([idx, idx + 1])
            if r < n_side - 1:
                edges.append([idx, idx + n_side])
    return grid, np.asarray(edges, dtype=int)

In [90]:
# Parameters
n_side = 16
alpha = 0.07
beta = 0.45
max_iter = 16
max_step = 0.5
grid, edges = build_square_grid(n_side)
pois = np.array([
    [0.80, 0.20],
    [0.20, 0.80],
    [0.60, 0.60],
    [0.40, 0.40],
    [0.10, 0.30]
])

In [91]:
frames_dir = "images/frames"
if os.path.exists(frames_dir):
    shutil.rmtree(frames_dir)
os.makedirs(frames_dir, exist_ok=True)

In [92]:
# Manual choromorph with frame export
g = grid.copy()
N = g.shape[0]
neighbors = [[] for _ in range(N)]
for a, b in edges:
    neighbors[a].append(b)
    neighbors[b].append(a)

images = []
for it in range(max_iter):
    move = np.zeros_like(g)
    for i, n in enumerate(g):
        diffs = pois - n
        dists = np.linalg.norm(diffs, axis=1)
        j = np.argmin(dists)
        d = dists[j]
        v_poi = alpha * d * (diffs[j] / d) if d > 0 else np.zeros(2)
        v_coh = np.zeros(2)
        if neighbors[i]:
            centroid = g[neighbors[i]].mean(axis=0)
            v_coh = beta * (centroid - n)
        v = v_poi + v_coh
        norm_v = np.linalg.norm(v)
        if norm_v > max_step:
            v *= max_step / norm_v
        move[i] = v

    for label, show_vectors in zip(['1_grid', '2_vectors', '3_updated'], [False, True, False]):
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.add_collection(LineCollection(g[edges], colors='lightgray', linewidths=1.0, alpha=0.5))
        ax.scatter(g[:, 0], g[:, 1], s=8, color='tab:blue', alpha=0.5, label='Nodes')
        ax.scatter(pois[:, 0], pois[:, 1], s=90, marker='*', color='tab:red', label='POIs')
        if show_vectors:
            ax.quiver(g[:, 0], g[:, 1], move[:, 0], move[:, 1],
                      angles='xy', scale_units='xy', scale=1, alpha=0.7,
                      color='tab:orange', width=0.003, label='Total move')
        ax.set_aspect('equal')
        ax.set_xlim(grid[:, 0].min() - 0.1, grid[:, 0].max() + 0.1)
        ax.set_ylim(grid[:, 1].min() - 0.1, grid[:, 1].max() + 0.1)
        ax.set_title(f"Step {it+1}: {label.replace('_', ' ').title()}")
        plt.tight_layout()
        frame_path = os.path.join(frames_dir, f"frame_{it:02d}_{label}.png")
        plt.savefig(frame_path)
        plt.close()
        images.append(imageio.v2.imread(frame_path))

    g += move

In [93]:
# Save the animation
gif_path = "images/choromorph_steps.gif"
imageio.mimsave(gif_path, images, duration=0.8)
print(f"GIF saved to {gif_path}")

GIF saved to choromorph_steps.gif
